In [5]:
from functions.primary_pipeline import *
from time import ctime
from tqdm import tqdm

In [2]:
time1 = ctime()
print(ctime())

Mon Sep 11 20:07:59 2023


In [3]:
# ### Import IBD Notes Dataframe ###

IBD_notes = pd.read_csv('M:Conner/Sayuma tsv/IBD_notes_2021_03_03.tsv', delimiter = '\t')


In [4]:
#searches through IBD_notes df to detect if note is a Nutrition Services Note

NS_search = IBD_notes['note'].apply(lambda x: Nutrition_services(x))
#creates an updated df w/o Nutrition Services notes
IBD_Fecal_Blood = IBD_notes.drop(NS_search[NS_search == 1].index).reset_index(drop = True)


In [5]:
# ### Detect HPI/ROS Questionnaire Presence ###


#creates column of HPI/ROS questionnaire Presence
IBD_Fecal_Blood['HPI/ROS Presence'] = IBD_Fecal_Blood['note'].apply(lambda x : search_ROS(x))


#creates column of HPI/ROS questionnaire date
IBD_Fecal_Blood['ROS_date'] = IBD_Fecal_Blood['note'].apply(lambda x : get_ROS_date(x))



In [6]:
# converts ROS_dates and deid service dates to datetime format:
IBD_Fecal_Blood['ROS_date'] = pd.to_datetime(IBD_Fecal_Blood['ROS_date'])
IBD_Fecal_Blood['deid_service_date_cdw'] = pd.to_datetime(IBD_Fecal_Blood['deid_service_date_cdw'])


In [7]:
ROS_days_between = []
# calculates the difference between ROS date and service date, NA rows appended to value 999
for i in range(len(IBD_Fecal_Blood['ROS_date'])):
    if IBD_Fecal_Blood['ROS_date'].iloc[i] is not pd.NaT:
        days_between = IBD_Fecal_Blood['deid_service_date_cdw'].iloc[i] - IBD_Fecal_Blood['ROS_date'].iloc[i]
        ROS_days_between.append(days_between.days)
    else:
        ROS_days_between.append(999)

In [8]:
# ### Create List of Valid ROS questionaire ###

#creates array of True/False values corresponding to whether or not ROS is valid and present
ROS_valid = []
for delta in ROS_days_between:
    ROS_valid.append(valid_ROS(delta))
    
    
IBD_Fecal_Blood['ROS_Valid'] = ROS_valid


In [9]:
# ### Grabbing Fecal Blood Input from Valid ROS Questionaires ###


# grab all note id's which correspond to valid ROS
Valid_note_ids = IBD_Fecal_Blood[IBD_Fecal_Blood['ROS_Valid'] == True]['deid_note_id']



In [10]:
ROS_answers = {}
count = 0
for ids in tqdm(Valid_note_ids):
    count += 1 
    #print(count)
    index = IBD_Fecal_Blood[IBD_Fecal_Blood['deid_note_id'] == ids].index.values[0]
    ROS_answers[ids] = get_fecal_answer(IBD_Fecal_Blood['note'][index])

100%|████████████████████████████████| 1542/1542 [10:59<00:00,  2.34it/s]


In [11]:

# create a list of indexes for ROS where the parser did not find an answer to the blood in stool question
None_list = []
ROS_keys = ROS_answers.keys()

for ids in ROS_keys:
    if ROS_answers[ids] == None:
        # deletes dictionary none value
        ROS_answers = removekey(ROS_answers, ids)
        
# ### Making Fecal_Blood_Value Column ####


shape = (len(IBD_Fecal_Blood), 1)
IBD_Fecal_Blood['Fecal_Blood_Value'] = np.ones(shape)

# ### Adding ROS/HPI Answers to Fecal_Blood_Value Column ###

for key in ROS_answers.keys():
    index = get_index(key, IBD_Fecal_Blood)
    IBD_Fecal_Blood.loc[index, 'Fecal_Blood_Value'] = ROS_answers[key]

In [12]:
IBD_Fecal_Blood.columns

Index(['Unnamed: 0', '_id', 'deid_PatientDurableKey', 'deid_note_key',
       'deid_note_id', 'note_type', 'deid_encounter_id_cdw',
       'deid_EncounterKey', 'enc_dept_specialty', 'prov_specialty',
       'deid_service_date_cdw', 'note', 'DateKeyValue', 'BirthDate', 'Age',
       'crohns', 'ulc_colitis', 'HPI/ROS Presence', 'ROS_date', 'ROS_Valid',
       'Fecal_Blood_Value'],
      dtype='object')

In [13]:
# ### Create HPI Column ###


#apply HPI extraction to each note
IBD_Fecal_Blood[['extracted_HPI', 'HPI_extraction_offset']] = IBD_Fecal_Blood.apply(lambda x : extract_HPI(x['note']), axis = 1 , result_type = 'expand')

IBD_Fecal_Blood['HPI'] = IBD_Fecal_Blood['extracted_HPI'].apply(lambda x : clean_tail(x))


In [14]:
# ### Remove Rows which Failed HPI Extraction ###


# converts every None value in HPI column to 1 for the purposes of removal
for index, row in enumerate(IBD_Fecal_Blood['HPI']):
    if row is None:
        IBD_Fecal_Blood.loc[index, 'HPI'] = 1

In [15]:
# ### Find Interval History ###

IBD_Fecal_Blood['Interval_History'] = IBD_Fecal_Blood.apply(lambda x: find_IntervalHistory2(x['HPI'], 
                                                                                            x['HPI_extraction_offset']), axis=1)

In [16]:
# ### Create Previous Note ID Column ###


# create a previous note id column for all remaining rows
IBD_Fecal_Blood['Previous_note_id'] = IBD_Fecal_Blood.apply(lambda x : find_previous_note(x['deid_note_id'], IBD_Fecal_Blood), axis = 1)

IBD_Fecal_Blood.to_json('data/ibd_notes.json')

In [17]:
# ### Preparing Dataframes for Previous Note Comparison ###


Previous_Note_df = IBD_Fecal_Blood[['HPI','HPI_extraction_offset', 'deid_PatientDurableKey', 'deid_note_id', 'Previous_note_id']]


Previous_Note_df = Previous_Note_df[Previous_Note_df['Previous_note_id'] != 1]

In [6]:
from functions.prev_note_pipeline import *
tqdm.pandas()

C:\tools\Anaconda3\lib\site-packages\tqdm\std.py:701: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [19]:
nlpstan = stanza.Pipeline("en")

nlp = spacy.load("en_core_web_sm")

2023-09-11 20:26:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-09-11 20:26:55 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2023-09-11 20:26:55 INFO: Using device: cpu
2023-09-11 20:26:55 INFO: Loading: tokenize
2023-09-11 20:26:55 INFO: Loading: pos
2023-09-11 20:26:57 INFO: Loading: lemma
2023-09-11 20:26:57 INFO: Loading: constituency
2023-09-11 20:26:58 INFO: Loading: depparse
2023-09-11 20:27:00 INFO: Loading: sentiment
2023-09-11 20:27:02 INFO: Loading: ner
2023-09-11 20:27:04 INFO: Done loading processors!


In [20]:
Previous_note = Previous_Note_df[
    ["deid_note_id", "Previous_note_id", "HPI_extraction_offset"]
]

In [21]:
# ### Get HPI from each note

# creates two columns current_HPI and previous_HPI
Previous_note["current_HPI"] = Previous_note["deid_note_id"].apply(
    lambda x: get_HPI(IBD_Fecal_Blood, x)
)
Previous_note["previous_HPI"] = Previous_note["Previous_note_id"].apply(
    lambda x: get_HPI(IBD_Fecal_Blood, x)
)

Previous_note["note"] = Previous_note["deid_note_id"].apply(lambda x: get_note(x, IBD_Fecal_Blood))


C:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [22]:
# ### Applying Previous Note Extraction to each Note ###

Previous_note[["addition_ranges", "additions"]] = Previous_note.progress_apply(
    lambda x: find_additions(
        nlp, x["current_HPI"], x["previous_HPI"], x["HPI_extraction_offset"], x["note"]
    )
    if x["current_HPI"] != 1
    else np.nan,
    axis=1,
    result_type="expand",
)

100%|████████████████████████████████| 4138/4138 [20:56<00:00,  3.29it/s]


In [23]:
Previous_note.to_json('data/previous_note.json')

In [24]:
# ### Compile Dataframes ###
IBD_Fecal_Blood = pd.read_json('data/ibd_notes.json')
Previous_note = pd.read_json('data/previous_note.json')

#create an empty additions column
shape = (len(IBD_Fecal_Blood), 1)
IBD_Fecal_Blood['additions'] = np.ones(shape)


note_ids = Previous_note['deid_note_id']
notes = Previous_note['additions']

#gets a list of indices from note ids 
note_ids_index = note_ids.apply(lambda x : get_index(x, IBD_Fecal_Blood))

for index, note_index in enumerate(note_ids_index):
    IBD_Fecal_Blood.loc[note_index, 'additions'] = notes.iloc[index]

In [25]:
# ### Apply 'Current' Extraction to IBD_Fecal_Blood ###


IBD_Fecal_Blood['Current_Extract'] = IBD_Fecal_Blood['HPI'].apply(lambda x : find_Today(x) if type(x) == str else 1)


In [26]:
# ### Applying Find_Blood Function ###


# applies find_Blood function to Interval History sections
Blood_Indicator_Interval_History = IBD_Fecal_Blood['Interval_History'].apply(lambda x : find_Blood(x) if x != 1 else 1)

# applies find_Blood function to HPI sections
Blood_Indicator_HPI = IBD_Fecal_Blood['HPI'].apply(lambda x : find_Blood(x) if x != 1 else 1)


#applies find_Blood function to Addition section
Blood_Indicator_Additions = IBD_Fecal_Blood['additions'].apply(lambda x : find_Blood(x) if x != 1 else 1)

#creates list of note Ids
Note_Ids = IBD_Fecal_Blood['deid_note_id']


In [27]:
# ### Make Blood_Indicator Dataframe ###


Blood_Indicator_zip = list(zip(Note_Ids, IBD_Fecal_Blood['HPI'], 
                               IBD_Fecal_Blood['Interval_History'], 
                               IBD_Fecal_Blood['additions'], 
                               Blood_Indicator_Interval_History,
                               Blood_Indicator_Additions, 
                               IBD_Fecal_Blood['Fecal_Blood_Value'],
                               Blood_Indicator_HPI))


Blood_Indicator_df = pd.DataFrame(Blood_Indicator_zip, columns = ['deid_note_id', 'HPI','Interval_History',
                                                                  'Additions', 'Blood_Indicator_Interval', 
                                                                  'Blood_Indicator_Addition', 'Fecal_Blood_Value',
                                                                  'Blood_Indicator_HPI'])


In [28]:
from functions.blood_pipeline import *
# from functions.diarrhea_pipeline import *

In [29]:
nlp_abdominal = return_abdominal_spacy()
nlp_abdominal = add_pipe(nlp_abdominal)

In [30]:
IBD_Fecal_Blood[['blood_mentions', 
                 'blood_mention_keyword', 
                 'blood_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_blood( x['Interval_History'] ),
                                                                  axis = 1, result_type = 'expand')

IBD_Fecal_Blood[['pain_mentions', 
                 'pain_mention_keyword', 
                 'pain_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_pain(x['Interval_History'], nlp_abdominal), 
                                                                 axis = 1, result_type = 'expand')

IBD_Fecal_Blood[['cr_mentions',
                 'cr_mention_keyword',
                 'cr_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_clinical_remission(x['Interval_History']),
                                                               axis = 1, result_type = 'expand')

IBD_Fecal_Blood[['well_mentions',
                 'well_mention_keyword',
                 'well_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_no_symptoms(x['Interval_History']),
                                                                 axis = 1, result_type = 'expand')

100%|██████████████████████████████| 7533/7533 [00:02<00:00, 3618.10it/s]


In [31]:
IBD_Fecal_Blood['Fecal_Blood_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['blood_mentions'], 
                                                                                                      'BLOOD', 
                                                                                                      nlp_abdominal, 
                                                                                                      x['deid_note_id']), axis = 1 )

IBD_Fecal_Blood['Pain_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['pain_mentions'], 
                                                                                               'PAIN', 
                                                                                               nlp_abdominal,  
                                                                                               x['deid_note_id']), axis = 1 ) 

IBD_Fecal_Blood['CR_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['cr_mentions'], 
                                                                                             'Clin_Rem', 
                                                                                             nlp_abdominal, 
                                                                                             x['deid_note_id']), axis = 1 )

IBD_Fecal_Blood['Well_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['well_mentions'], 
                                                                                               'WELL', 
                                                                                               nlp_abdominal, 
                                                                                               x['deid_note_id']), axis = 1 ) 



100%|█████████████████████████████| 7533/7533 [00:00<00:00, 16009.44it/s]


In [32]:
# ### Apply Blood Mention Extraction to Previous_Note Extraction ###

Previous_note['additions'] = Previous_note['additions'].apply(lambda x : convert_list_string(x))

Previous_note[['blood_mentions', 
               'blood_mention_keyword', 
               'blood_spans']] = Previous_note.progress_apply(lambda x : find_blood(x['additions']), 
                                                              axis = 1, result_type = 'expand')

Previous_note[['pain_mentions', 
               'pain_mention_keyword', 
               'pain_spans']] = Previous_note.progress_apply(lambda x : find_pain(x['additions'], nlp_abdominal), 
                                                             axis = 1, result_type = 'expand')

Previous_note[['cr_mentions', 
               'cr_mention_keyword', 
               'cr_spans']] = Previous_note.progress_apply(lambda x : find_clinical_remission(x['additions']), 
                                                           axis = 1, result_type = 'expand')

Previous_note[['well_mentions', 
               'well_mention_keyword', 
               'well_spans']] = Previous_note.progress_apply(lambda x : find_no_symptoms(x['additions']), 
                                                             axis = 1, result_type = 'expand')

Previous_note['Fecal_Blood_Present'] =  Previous_note.progress_apply(lambda x : get_tense_negation(x['blood_mentions'], 
                                                                                                   'BLOOD',
                                                                                                   nlp_abdominal, 
                                                                                                   x['deid_note_id']), axis = 1 )

Previous_note['Pain_Present'] = Previous_note.progress_apply(lambda x : get_tense_negation(x['pain_mentions'], 
                                                                                           'PAIN', nlp_abdominal,
                                                                                           x['deid_note_id']), axis = 1 )

Previous_note['CR_Present'] = Previous_note.progress_apply(lambda x : get_tense_negation(x['cr_mentions'], 
                                                                                         'Clin_Rem', nlp_abdominal,
                                                                                         x['deid_note_id']), axis = 1 )

Previous_note['Well_Present'] = Previous_note.progress_apply(lambda x : get_tense_negation(x['well_mentions'], 
                                                                                           'WELL', 
                                                                                           nlp_abdominal,
                                                                                           x['deid_note_id']), axis = 1 )

IBD_Fecal_Blood['Fecal_Blood_Value'] = IBD_Fecal_Blood['Fecal_Blood_Value'].apply(lambda x : encode(x))

IBD_Fecal_Blood['Previous_note_id'] = IBD_Fecal_Blood.apply(lambda x : find_previous_note(x['deid_note_id'], 
                                                                                          IBD_Fecal_Blood), axis = 1)


100%|████████████████████████████████| 4138/4138 [01:50<00:00, 37.51it/s]


In [33]:
# ### Make Blank Columns ###

#IBD_Fecal_Blood['blood_mention_c_takes'] = np.nan
IBD_Fecal_Blood['blood_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['blood_mention_Interval_History'] = np.nan
IBD_Fecal_Blood['pain_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['pain_mention_Interval_History'] = np.nan
IBD_Fecal_Blood['cr_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['cr_mention_Interval_History'] = np.nan
IBD_Fecal_Blood['well_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['well_mention_Interval_History'] = np.nan

# ### Encode Each Column ###
encode_column(IBD_Fecal_Blood, Previous_note, 'Fecal_Blood_Present', 'blood_mention_Previous_note', False)
encode_column(IBD_Fecal_Blood, Previous_note, 'Pain_Present', 'pain_mention_Previous_note', False)
encode_column(IBD_Fecal_Blood, Previous_note, 'CR_Present', 'cr_mention_Previous_note', False)
encode_column(IBD_Fecal_Blood, Previous_note, 'Well_Present', 'well_mention_Previous_note', False)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Fecal_Blood_Present', 'blood_mention_Interval_History', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Pain_Present', 'pain_mention_Interval_History', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'CR_Present', 'cr_mention_Interval_History', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Well_Present', 'well_mention_Interval_History', True)

In [34]:
IBD_Fecal_Blood['HPI_range'] = IBD_Fecal_Blood.apply(lambda x : get_range(x['HPI_extraction_offset'], 
                                                                          x['HPI']) if x['HPI'] != 1 else 1, axis = 1)

# ### Applying Blood Mention Extraction to Remaining Notes ###
IBD_Fecal_Blood['Interval_History'] = IBD_Fecal_Blood['Interval_History'].apply(lambda x : x if x != 1 else np.nan)

#capture remaining notes
remaining_notes = IBD_Fecal_Blood[IBD_Fecal_Blood['Interval_History'].isna()]
remaining_notes = remaining_notes[remaining_notes['Previous_note_id'].isna()]
remaining_notes = remaining_notes[remaining_notes['Fecal_Blood_Value'].isna()] 

remaining_notes_pain = IBD_Fecal_Blood[IBD_Fecal_Blood['Interval_History'].isna()]
remaining_notes_pain = remaining_notes_pain[remaining_notes_pain['Previous_note_id'].isna()]

In [35]:
# #### Blood Mention Extraction Remaining Notes #####

IBD_Fecal_Blood[['blood_mentions_HPI', 
                 'blood_mentions_remaining_keywords', 
                 'blood_spans']] = IBD_Fecal_Blood.apply(lambda x : find_blood(x['HPI']), result_type = 'expand', axis = 1  )

IBD_Fecal_Blood[['blood_mentions_HPI', 
                 'blood_mentions_remaining_keywords']] = IBD_Fecal_Blood.progress_apply(lambda x : get_mention_location(x['HPI'],
                                                                                                                        x['blood_mentions_HPI'],
                                                                                                                        x['blood_spans'], 
                                                                                                                        x['blood_mentions_remaining_keywords']),
                                                                                        result_type = 'expand', axis = 1 )

IBD_Fecal_Blood[['pain_mentions_HPI', 
                 'pain_mentions_remaining_keywords', 
                 'pain_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_pain(x['HPI'], nlp_abdominal), 
                                                                 result_type = 'expand', axis = 1  )

IBD_Fecal_Blood[['pain_mentions_HPI_location', 
                 'pain_mentions_remaining_keywords']] = IBD_Fecal_Blood.progress_apply(lambda x : get_mention_location(x['HPI'], 
                                                                                                                       x['pain_mentions_HPI'], 
                                                                                                                       x['pain_spans'], 
                                                                                                                       x['pain_mentions_remaining_keywords']), 
                                                                                       result_type = 'expand', axis = 1 )

IBD_Fecal_Blood[['cr_mentions_HPI', 
                 'cr_mention_keyword',
                 'cr_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_clinical_remission(x['HPI']), 
                                                               axis = 1, result_type = 'expand')

IBD_Fecal_Blood[['cr_mentions_HPI_location', 
                 'cr_mention_keyword']] = IBD_Fecal_Blood.progress_apply(lambda x : get_mention_location(x['HPI'],
                                                                                                         x['cr_mentions'], 
                                                                                                         x['cr_spans'], 
                                                                                                         x['cr_mention_keyword']),
                                                                         axis = 1, result_type = 'expand')

IBD_Fecal_Blood[['well_mentions_HPI',
                 'well_mention_keyword',
                 'well_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_no_symptoms(x['HPI']), 
                                                                 axis = 1, result_type = 'expand')

IBD_Fecal_Blood[['well_mentions_HPI_location', 
                 'well_mention_keyword']] = IBD_Fecal_Blood.progress_apply(lambda x : get_mention_location(x['HPI'], 
                                                                                                           x['well_mentions'],
                                                                                                           x['well_spans'], 
                                                                                                           x['well_mention_keyword']),
                                                                           axis = 1, result_type = 'expand')

IBD_Fecal_Blood['Pain_Present_Remaining'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['pain_mentions_HPI'], 
                                                                                                        'PAIN', 
                                                                                                        nlp_abdominal, 
                                                                                                        x['deid_note_id']), axis = 1)

IBD_Fecal_Blood['Fecal_Blood_Present_HPI'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['blood_mentions_HPI'],
                                                                                                          'BLOOD', 
                                                                                                          nlp_abdominal, 
                                                                                                          x['deid_note_id']) if x['blood_mentions_HPI'] != 1 else np.nan, axis = 1)

IBD_Fecal_Blood['Pain_Present_HPI'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['pain_mentions_HPI_location'], 
                                                                                                   'PAIN', 
                                                                                                   nlp_abdominal, 
                                                                                                   x['deid_note_id']), axis = 1)

IBD_Fecal_Blood['CR_Present_HPI'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['cr_mentions_HPI_location'], 
                                                                                                 'Clin_Rem', 
                                                                                                 nlp_abdominal,
                                                                                                 x['deid_note_id']), axis = 1)

IBD_Fecal_Blood['Well_Present_HPI'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['well_mentions_HPI_location'], 
                                                                                                   'WELL',
                                                                                                   nlp_abdominal, 
                                                                                                   x['deid_note_id']), axis = 1)

100%|█████████████████████████████| 7533/7533 [00:00<00:00, 19737.77it/s]


In [36]:
# #### Encode Results ####
IBD_Fecal_Blood['pain_mention_remaining_notes'] = np.nan
IBD_Fecal_Blood['blood_mention_HPI'] = np.nan
IBD_Fecal_Blood['pain_mention_HPI'] = np.nan
IBD_Fecal_Blood['cr_mention_HPI'] = np.nan
IBD_Fecal_Blood['well_mention_HPI'] = np.nan
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Pain_Present_Remaining', 'pain_mention_remaining_notes', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Fecal_Blood_Present_HPI', 'blood_mention_HPI', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Pain_Present_HPI', 'pain_mention_HPI', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'CR_Present_HPI', 'cr_mention_HPI', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Well_Present_HPI', 'well_mention_HPI', True)


In [37]:
# ### Consolidate Each Blood Mention into a single Column ###
IBD_Fecal_Blood['Fecal_Blood_Value_Master'] = np.nan
IBD_Fecal_Blood['Pain_Value_Master'] = np.nan
IBD_Fecal_Blood['Pain_ROS'] = np.nan
IBD_Fecal_Blood['CR_Master'] = np.nan
IBD_Fecal_Blood['CR_ROS'] = np.nan
IBD_Fecal_Blood['Well_Master'] = np.nan
IBD_Fecal_Blood['Well_ROS'] = np.nan


consolidate_columns(row_names_blood, 'Fecal_Blood_Value_Master', IBD_Fecal_Blood)
consolidate_columns(row_names, 'Pain_Value_Master', IBD_Fecal_Blood)
consolidate_columns(rows_names_cr, 'CR_Master', IBD_Fecal_Blood)
consolidate_columns(row_names_well, 'Well_Master', IBD_Fecal_Blood)


[]

In [38]:
print(f'Start: {time1}, End: {ctime()}')

Start: Mon Sep 11 20:07:59 2023, End: Tue Sep 12 00:38:42 2023


In [39]:
IBD_Fecal_Blood.to_json('data/abdominal_results.json')

## Creating the Diarrhea Portion

In [1]:
from functions.diarrhea_pipeline import *

In [2]:
nlp_diarrhea = return_diarrhea_spacy()
nlp_diarrhea = add_pipes(nlp_diarrhea)

In [3]:
IBD_Fecal_Blood = pd.read_json('data/ibd_notes.json')
Previous_note = pd.read_json('data/previous_note.json')

In [7]:
IBD_Fecal_Blood[['dia_mentions', 
                 'dia_mention_keyword', 
                 'dia_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_diarrhea( x['Interval_History'] ), 
                                                                axis = 1, result_type = 'expand')

IBD_Fecal_Blood['stool_fre_mentions'] = IBD_Fecal_Blood.progress_apply(lambda x : find_stool_freq(x['Interval_History']),
                                                                       axis = 1 )

IBD_Fecal_Blood[['cr_mentions', 
                 'cr_mention_keyword',
                 'cr_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_clinical_remission(x['Interval_History']), 
                                                               axis = 1, result_type = 'expand')

IBD_Fecal_Blood[['well_mentions', 
                 'well_mention_keyword', 
                 'well_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_no_symptoms(x['Interval_History']),
                                                                 axis = 1, result_type = 'expand')

IBD_Fecal_Blood['Diarrhea_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['dia_mentions'],
                                                                                                   'Diarrhea', 
                                                                                                   nlp_diarrhea,
                                                                                                   x['deid_note_id']), axis = 1 )

IBD_Fecal_Blood['stool_range_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['stool_fre_mentions'],
                                                                                                      'range', 
                                                                                                      nlp_diarrhea,
                                                                                                      x['deid_note_id']), axis = 1)

IBD_Fecal_Blood['Bristol_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['stool_fre_mentions'], 
                                                                                                  'Bristol', 
                                                                                                  nlp_diarrhea, 
                                                                                                  x['deid_note_id']), axis = 1)

IBD_Fecal_Blood['CR_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['cr_mentions'], 
                                                                                             'Clin_Rem', 
                                                                                             nlp_diarrhea,
                                                                                             x['deid_note_id']), axis = 1 ) 

IBD_Fecal_Blood['Well_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['well_mentions'], 
                                                                                               'WELL',
                                                                                               nlp_diarrhea,
                                                                                               x['deid_note_id']), axis = 1 )

100%|█████████████████████████████| 7533/7533 [00:00<00:00, 18450.86it/s]


In [8]:
Previous_note['HPI'] = Previous_note['deid_note_id'].map(IBD_Fecal_Blood.set_index('deid_note_id')['HPI'])

Previous_note[['dia_mentions', 
               'dia_mention_keyword', 
               'dia_spans']] = Previous_note.progress_apply(lambda x : find_diarrhea( x['additions'] ) if x['additions'] != 1 else find_diarrhea(x['HPI']), 
                                                            axis = 1, result_type = 'expand')

100%|████████████████████████████████| 4138/4138 [04:06<00:00, 16.78it/s]


In [9]:
Previous_note['stool_fre_mentions'] = Previous_note.progress_apply(lambda x : find_stool_freq(x['additions']) if x['additions'] != 1 else find_stool_freq(x['HPI']), 
                                                                   axis = 1 )

Previous_note[['cr_mentions', 
               'cr_mention_keyword',
               'cr_spans']] = Previous_note.progress_apply(lambda x : find_clinical_remission(x['additions']) if x['additions'] != 1 else find_clinical_remission(x['HPI']), 
                                                           axis = 1, result_type = 'expand')

Previous_note[['well_mentions', 
               'well_mention_keyword',
               'well_spans']] = Previous_note.progress_apply(lambda x : find_no_symptoms(x['additions']) if x['additions'] != 1 else find_no_symptoms(x['HPI']), 
                                                             axis = 1, result_type = 'expand')

Previous_note['Diarrhea_Present'] = Previous_note.progress_apply(lambda x : get_tense_negation(x['dia_mentions'], 
                                                                                               'Diarrhea', 
                                                                                               nlp_diarrhea, 
                                                                                               x['deid_note_id']), axis = 1 )

Previous_note['stool_range_Present'] = Previous_note.progress_apply(lambda x : get_tense_negation(x['stool_fre_mentions'],
                                                                                                  'range', 
                                                                                                  nlp_diarrhea,
                                                                                                  x['deid_note_id']), axis = 1 )

Previous_note['Bristol_Present'] = Previous_note.progress_apply(lambda x : get_tense_negation(x['stool_fre_mentions'], 
                                                                                              'Bristol', 
                                                                                              nlp_diarrhea, 
                                                                                              x['deid_note_id']), axis = 1)


100%|████████████████████████████████| 4138/4138 [00:50<00:00, 81.54it/s]


In [10]:
Previous_note['CR_Present'] = Previous_note.progress_apply(lambda x : get_tense_negation(x['cr_mentions'], 
                                                                                         'Clin_Rem', 
                                                                                         nlp_diarrhea, 
                                                                                         x['deid_note_id']), axis = 1 )

Previous_note['Well_Present'] = Previous_note.progress_apply(lambda x : get_tense_negation(x['well_mentions'], 
                                                                                           'WELL', 
                                                                                           nlp_diarrhea, 
                                                                                           x['deid_note_id']), axis = 1 )

100%|████████████████████████████████| 4138/4138 [00:46<00:00, 88.23it/s]


In [11]:
IBD_Fecal_Blood['dia_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['dia_mention_Interval_History'] = np.nan

IBD_Fecal_Blood['stool_fre_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['stool_fre_mention_Interval_History'] = np.nan

IBD_Fecal_Blood['bristol_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['bristol_mention_Interval_History'] = np.nan

IBD_Fecal_Blood['CR_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['CR_mention_Interval_History'] = np.nan

IBD_Fecal_Blood['well_mention_Previous_note'] = np.nan
IBD_Fecal_Blood['well_mention_Interval_History'] = np.nan

In [12]:
encode_column(IBD_Fecal_Blood, Previous_note, 'Diarrhea_Present', 'dia_mention_Previous_note', False)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Diarrhea_Present', 'dia_mention_Interval_History', True)
encode_column(IBD_Fecal_Blood, Previous_note, 'stool_range_Present', 'stool_fre_mention_Previous_note', False)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'stool_range_Present', 'stool_fre_mention_Interval_History', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Bristol_Present', 'bristol_mention_Interval_History', True)
encode_column(IBD_Fecal_Blood, Previous_note, 'Bristol_Present', 'bristol_mention_Previous_note', False)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Well_Present', 'well_mention_Interval_History', True)
encode_column(IBD_Fecal_Blood, Previous_note, 'Well_Present', 'well_mention_Previous_note', False)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'CR_Present', 'CR_mention_Interval_History', True)
encode_column(IBD_Fecal_Blood, Previous_note, 'CR_Present', 'CR_mention_Previous_note', False)

In [14]:
IBD_Fecal_Blood['Previous_note_id'] = IBD_Fecal_Blood.apply(lambda x : find_previous_note(x['deid_note_id'], 
                                                                                          IBD_Fecal_Blood), axis = 1)

IBD_Fecal_Blood['Interval_History'] = IBD_Fecal_Blood['Interval_History'].apply(lambda x : x if x != 1 else np.nan)

IBD_Fecal_Blood['HPI_range'] = IBD_Fecal_Blood.apply(lambda x : get_range(x['HPI_extraction_offset'], 
                                                                          x['HPI']) if x['HPI'] != 1 else 1, axis = 1)

IBD_Fecal_Blood[['dia_mentions_HPI', 
                 'dia_mentions_HPI_keywords', 
                 'dia_HPI_spans']] = IBD_Fecal_Blood.apply(lambda x : find_diarrhea(x['HPI']), 
                                                           result_type = 'expand', axis = 1  )

In [15]:
IBD_Fecal_Blood[['dia_mentions_HPI', 
                 'dia_mentions_HPI_keywords',]] = IBD_Fecal_Blood.progress_apply(lambda x : get_mention_location(x['HPI'], 
                                                                                                                 x['dia_mentions_HPI'], 
                                                                                                                 x[ 'dia_HPI_spans'], 
                                                                                                                 x['dia_mentions_HPI_keywords']),
                                                                                 result_type = 'expand', axis = 1 )

IBD_Fecal_Blood['stool_fre_mentions_HPI'] = IBD_Fecal_Blood.progress_apply(lambda x : find_stool_freq(x['HPI']), axis = 1 )

IBD_Fecal_Blood[['cr_mentions_HPI', 
                 'cr_mention_HPI_keyword',
                 'cr_HPI_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_clinical_remission(x['HPI']), 
                                                                   axis = 1, result_type = 'expand')

IBD_Fecal_Blood[['well_mentions_HPI', 
                 'well_mention_HPI_keyword',
                 'well_HPI_spans']] = IBD_Fecal_Blood.progress_apply(lambda x : find_no_symptoms(x['HPI']), 
                                                                     axis = 1, result_type = 'expand')

IBD_Fecal_Blood['Dia_Present_Remaining'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['dia_mentions_HPI'], 
                                                                                                        'Diarrhea', 
                                                                                                        nlp_diarrhea, 
                                                                                                        x['deid_note_id']), axis = 1)

IBD_Fecal_Blood['stool_range_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['stool_fre_mentions_HPI'], 
                                                                                                      'range', 
                                                                                                      nlp_diarrhea,
                                                                                                      x['deid_note_id']), axis = 1)

IBD_Fecal_Blood['Bristol_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['stool_fre_mentions_HPI'], 
                                                                                                  'Bristol', 
                                                                                                  nlp_diarrhea, 
                                                                                                  x['deid_note_id']), axis = 1)

100%|████████████████████████████████| 7533/7533 [02:41<00:00, 46.62it/s]


In [16]:
IBD_Fecal_Blood['CR_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['cr_mentions_HPI'], 
                                                                                             'Clin_Rem', 
                                                                                             nlp_diarrhea, 
                                                                                             x['deid_note_id']), axis = 1 ) 

IBD_Fecal_Blood['Well_Present'] = IBD_Fecal_Blood.progress_apply(lambda x : get_tense_negation(x['well_mentions_HPI'],
                                                                                               'WELL', 
                                                                                               nlp_diarrhea,
                                                                                               x['deid_note_id']), axis = 1 ) 

100%|████████████████████████████████| 7533/7533 [01:55<00:00, 65.38it/s]


In [17]:
IBD_Fecal_Blood['dia_mention_remaining_notes'] = np.nan
IBD_Fecal_Blood['stool_fre_mention_remaining_notes'] = np.nan
IBD_Fecal_Blood['bristol_mention_remaining_notes'] = np.nan

IBD_Fecal_Blood['CR_mention_remaining_notes'] = np.nan
IBD_Fecal_Blood['well_mention_remaining_notes'] = np.nan

In [18]:
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Dia_Present_Remaining', 'dia_mention_remaining_notes', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'stool_range_Present', 'stool_fre_mention_remaining_notes', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Bristol_Present', 'bristol_mention_remaining_notes', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'Well_Present', 'well_mention_remaining_notes', True)
encode_column(IBD_Fecal_Blood, IBD_Fecal_Blood, 'CR_Present', 'CR_mention_remaining_notes', True)

In [19]:
IBD_Fecal_Blood['Dia_ROS'] = np.nan
IBD_Fecal_Blood['Dia_Master'] = np.nan
IBD_Fecal_Blood['stool_fre_ROS'] = np.nan
IBD_Fecal_Blood['Stool_fre_Master'] = np.nan
IBD_Fecal_Blood['Bristol_ROS'] = np.nan
IBD_Fecal_Blood['Bristol_Master'] = np.nan
IBD_Fecal_Blood['CR_ROS'] = np.nan
IBD_Fecal_Blood['CR_Master'] = np.nan
IBD_Fecal_Blood['Well_ROS'] = np.nan
IBD_Fecal_Blood['Well_Master'] = np.nan

In [21]:
consolidate_columns(row_names_dia, 'Dia_Master', IBD_Fecal_Blood)
consolidate_columns(row_names_stool_fre, 'Stool_fre_Master', IBD_Fecal_Blood)
consolidate_columns(row_names_bristol, 'Bristol_Master', IBD_Fecal_Blood)
consolidate_columns(row_names_well, 'Well_Master', IBD_Fecal_Blood)
consolidate_columns(row_names_cr, 'CR_Master', IBD_Fecal_Blood)

[]

In [22]:
print(ctime())

Tue Sep 12 02:06:12 2023


In [23]:
IBD_Fecal_Blood.to_json('data/diarrhea_results.json')